In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


In [2]:
Crop_ds=pd.read_csv("crop_preprocess_ds.csv")
Crop_ds

,Crop_Year,Area_Hectares,Production_Tonnes,Rainfall_mm,Fertilizer_Kgms,Pesticide_Kgms,Yield/unitArea,Crop_Arhar/Tur,Crop_Bajra,Crop_Banana,...,State_Odisha,State_Puducherry,State_Punjab,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal
0,1997,73814.0,56708,2051.4,7024878.38,22882.34,0.796087,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1997,6637.0,4685,2051.4,631643.29,2057.47,0.710435,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1997,796.0,22,2051.4,75755.32,246.76,0.238333,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1997,19656.0,309194,2051.4,1870661.52,6093.36,5.118226,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1997,1739.0,794,2051.4,165500.63,539.09,0.420909,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19568,1998,4000.0,2000,1498.0,395200.00,1160.00,0.500000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19569,1998,1000.0,3000,1498.0,98800.00,290.00,3.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19570,1997,187823.5,309194,1356.2,25054187.10,49993.35,1.285000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19571,1997,187823.5,5488,1356.2,25054187.10,49993.35,0.016667,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
indep_X=Crop_ds.drop('Yield/unitArea',axis=1)
dep_Y = Crop_ds[['Yield/unitArea']]


In [4]:
dep_Y=np.ravel(dep_Y)

In [7]:
#Spliting & Standardization
def split_scalar(indep_X, dep_Y):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test
#PCA function
def pCA(X_train, X_test, n):
    pca = PCA(n_components=n)
    X_train_PCA = pca.fit_transform(X_train)
    X_test_PCA = pca.transform(X_test)
    explained_variance = pca.explained_variance_ratio_
    return pca, X_train_PCA, X_test_PCA, explained_variance
def r2_prediction(regressor,X_test,y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2
#SVM Linear function
def svm_linear(X_train, y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel='linear')
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return r2
def svm_NL(X_train, y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel='rbf')
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return r2 
def Decision(X_train, y_train, X_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return r2
#Random Forest Function
def random(X_train, y_train, X_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators=10, random_state=0)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return r2
def ridge(X_train, y_train, X_test):
    from sklearn.linear_model import Ridge
    regressor= Ridge(alpha = 0.5)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return r2
def PCA_ml( accsvml, accsvmnl, accdes, accrf,accrg):
    dataframe = pd.DataFrame(index=['n=12'], columns=['SVML', 'SVMNL', 'DecisionTree', 'RandomForest','RidgeRegression'])
    for number, idex in enumerate(dataframe.index):
        dataframe.loc[idex, "SVML"] = accsvml[number]
        dataframe.loc[idex, "SVMNL"]=accsvmnl[number]
        dataframe.loc[idex, "DecisionTree"]=accdes[number]
        dataframe.loc[idex, "RandomForest"]=accrf[number]
        dataframe.loc[idex, "RidgeRegression"]=accrg[number]
    return dataframe    

In [8]:
X_train, X_test, y_train, y_test = split_scalar(indep_X, dep_Y)


In [9]:
# PCA
pca, X_train_PCA, X_test_PCA, explained_variance = pCA(X_train, X_test,12)

# Initialize lists to store accuracies
accsvml = []
accsvmnl = []
accdes = []
accrf = []
accrg=[]


In [10]:
    r2_sl = svm_linear(X_train_PCA, y_train, X_test_PCA)
    accsvml.append(r2_sl)
    
    r2_NL = svm_NL(X_train_PCA,y_train, X_test_PCA)  
    accsvmnl.append(r2_NL)
    
    r2_d = Decision(X_train_PCA, y_train, X_test_PCA)  
    accdes.append(r2_d)
    
    r2_r = random(X_train_PCA, y_train, X_test_PCA)
    accrf.append(r2_r)

    r2_rg=ridge(X_train_PCA,y_train,X_test_PCA)
    accrg.append(r2_rg)

    result = PCA_ml(accsvml, accsvmnl, accdes, accrf,accrg)

In [11]:
X_train_PCA.shape

(14679, 12)

In [12]:
X_test_PCA

array([[-0.60813061,  0.49708571, -0.17387745, ..., -0.88495286,
        -1.10270793, -0.14017161],
       [ 1.44214541,  2.71011675, -0.08289177, ..., -0.54036157,
        -0.71970892,  1.80992256],
       [-0.96326765, -1.03662323,  2.20517795, ..., -0.01417378,
        -1.80493537,  0.14530944],
       ...,
       [-1.36507408,  1.35322345,  0.24854265, ..., -0.56922267,
         0.12039435,  0.4193981 ],
       [ 4.18176678,  1.19594488, -0.15656057, ...,  0.24125479,
         0.32015803,  0.57124046],
       [-1.47833543,  1.66419883, -2.09082184, ...,  0.7281147 ,
        -1.8213738 , -0.06453572]])

In [13]:
X_test_PCA.shape


(4894, 12)

In [14]:
result

,SVML,SVMNL,DecisionTree,RandomForest,RidgeRegression
n=12,0.290134,0.74647,0.658832,0.858172,0.329682


In [16]:
def grid_search_models(X_train_PCA, y_train):
    svm_param_grid = {
        'C': [10, 100],
        'kernel': ['rbf', 'poly', 'sigmoid']
    }
    
    # Define parameter grid for RandomForestRegressor
    rf_param_grid = {
        'criterion': ['mse', 'mae'],
        'max_features': ['auto', 'sqrt', 'log2'],
        'n_estimators': [10, 100]
    }

    # Define parameter grid for DecisionTreeRegressor
    dt_param_grid = {
        'criterion': ['mse', 'mae'],
        'max_features': ['auto', 'sqrt', 'log2']
    }

    # Define parameter grid for Ridge Regression
    ridge_param_grid = {
        'alpha': [0.1, 1, 10],
        'solver': ['auto', 'svd', 'sag', 'saga']
    }
    
    svm_grid_search = GridSearchCV(SVR(), svm_param_grid, refit=True, verbose=3, n_jobs=-1)
    svm_grid_search.fit(X_train_PCA, y_train)

     # Perform grid search with cross-validation for RandomForestRegressor
    rf_grid_search = GridSearchCV(RandomForestRegressor(), rf_param_grid, refit=True, verbose=3, n_jobs=-1)
    rf_grid_search.fit(X_train_PCA, y_train)

    # Perform grid search with cross-validation for DecisionTreeRegressor
    dt_grid_search = GridSearchCV(DecisionTreeRegressor(), dt_param_grid, refit=True, verbose=3, n_jobs=-1)
    dt_grid_search.fit(X_train_PCA, y_train)

    # Perform grid search with cross-validation for Ridge Regression
    ridge_grid_search = GridSearchCV(Ridge(), ridge_param_grid, refit=True, verbose=3, n_jobs=-1)
    ridge_grid_search.fit(X_train_PCA, y_train)

    return {
        # Print best parameters and score for SVM Non-linear
        "SVM Non-linear":{
        "Best parameters": svm_grid_search.best_params_,
        "Best score":svm_grid_search.best_score_
        },
        "RandomForestRegressor": {
            "best_params": rf_grid_search.best_params_,
            "best_score": rf_grid_search.best_score_
        },
        "DecisionTreeRegressor": {
            "best_params": dt_grid_search.best_params_,
            "best_score": dt_grid_search.best_score_
        },
        "RidgeRegression": {
            "best_params": ridge_grid_search.best_params_,
            "best_score": ridge_grid_search.best_score_
        }
    }
    


In [17]:
result=grid_search_models(X_train_PCA, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 